# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import importlib
import api_keys
importlib.reload(api_keys)

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,alice springs,-23.7000,133.8833,34.75,14,7,10.29,AU,1696212217
1,1,grytviken,-54.2811,-36.5092,2.15,93,11,5.14,GS,1696212330
2,2,tekkali,18.6167,84.2333,27.53,86,99,2.60,IN,1696212330
3,3,zag,46.9441,99.1650,4.27,62,25,2.02,MN,1696212330
4,4,waitangi,-43.9535,-176.5597,14.34,88,65,3.13,NZ,1696212330


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    'Lng',
    'Lat',  
    geo=True,
    tiles="OSM",
    size='Humidity',
    color='Humidity', 
    hover_cols=['City', 'Humidity'],
    title='City Humidity Map',
    frame_width = 800,
    frame_height = 800
)

# Displaying the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities_df = city_data_df.loc[
    (city_data_df["Max Temp"] >= 20) & 
    (city_data_df["Max Temp"] <= 30) & 
    (city_data_df["Wind Speed"] < 10) & 
    (city_data_df["Cloudiness"] < 50)
]

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()
ideal_cities_df.reset_index(inplace=True, drop=True)

# Display sample data
ideal_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,5,noumea,-22.2763,166.4572,26.99,99,43,7.86,NC,1696211877
1,10,remire-montjoly,4.9167,-52.2667,27.02,83,0,2.06,GF,1696212331
2,12,saint-philippe,-21.3585,55.7679,21.77,75,0,2.58,RE,1696212331
3,17,evanston,42.0411,-87.6901,22.99,70,0,2.06,US,1696212201
4,30,hutchinson island south,27.2995,-80.2205,27.25,89,0,5.36,US,1696212333


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""
hotel_df.reset_index(inplace=True, drop=True)

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,noumea,NC,-22.2763,166.4572,99,
1,remire-montjoly,GF,4.9167,-52.2667,83,
2,saint-philippe,RE,-21.3585,55.7679,75,
3,evanston,US,42.0411,-87.6901,70,
4,hutchinson island south,US,27.2995,-80.2205,89,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [11]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "radius": radius,
    "type": "lodging",
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["categories"] = "accommodation.hotel"
    params["filter"] = f"circle:{lng},{lat},10000"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places?"
    
    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
noumea - nearest hotel: Hotel Gondwana
remire-montjoly - nearest hotel: Complexe Belova
saint-philippe - nearest hotel: Le Baril
evanston - nearest hotel: Margarita European Inn
hutchinson island south - nearest hotel: Vistana Beach Club 1
betioky - nearest hotel: Hotel Salema
warrnambool - nearest hotel: Central Court
mount gambier - nearest hotel: The Old Mount Gambier Gaol
georgetown - nearest hotel: Page 63 hostel
san patricio - nearest hotel: No hotel found
tokar - nearest hotel: No hotel found
new norfolk - nearest hotel: Woodbridge on the Derwent
west island - nearest hotel: Cocos Village Bungalows
port lincoln - nearest hotel: Boston Hotel
port mathurin - nearest hotel: Escale Vacances
ghadamis - nearest hotel: فندق عين الفارس
laguna - nearest hotel: Holiday Inn Express & Suites
grand gaube - nearest hotel: Hotel Lux- Grand Gaube
dakhla - nearest hotel: Hotel El Massira
kushiro - nearest hotel: ANA Hotel Kushiro
howard springs - nearest hotel: No hotel fou

,City,Country,Lat,Lng,Humidity,Hotel Name
0,noumea,NC,-22.2763,166.4572,99,Hotel Gondwana
1,remire-montjoly,GF,4.9167,-52.2667,83,Complexe Belova
2,saint-philippe,RE,-21.3585,55.7679,75,Le Baril
3,evanston,US,42.0411,-87.6901,70,Margarita European Inn
4,hutchinson island south,US,27.2995,-80.2205,89,Vistana Beach Club 1
...,...,...,...,...,...,...
116,kavieng,PG,-2.5744,150.7967,72,Kavieng Hotel
117,roccella ionica,IT,38.3220,16.4009,72,Mediterraneo
118,balkanabat,TM,39.5108,54.3671,20,Nebitçi
119,merizo village,GU,13.2658,144.6691,87,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [12]:
%%capture --no-display

# Create a hover tooltip to display the desired information
tooltip = [
    ('Hotel Name', '@{Hotel Name}'),
    ('Country', '@Country'),
    ('City', '@City')
]

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    'Lng',  
    'Lat',  
    geo=True, 
    tiles='OSM',
    color='Humidity', 
    hover_cols=['Hotel Name', 'Country'], 
    title='Hotel Locations',
    frame_height=500,
    frame_width=500,
    size='Humidity',
    legend_label='Humidity (%)',
    tooltip=tooltip
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,Hotel Name,Country)